In [1]:
using LinearAlgebra
using LoopVectorization
using LinearMaps
using ArnoldiMethod

function prepareDiag(L)
    diag = zeros(2^L)
    for state = 1:2^L
        for i = 1:L
            j = i==L ? 1 : i+1
            @inbounds diag[state] -= (((state >> (i-1))&1) == ((state >> (j-1))&1)) ? 1 : -1
        end
    end
    diag
end
    
function Hfunc!(C, B, diag, L)
    N = length(diag)
    @tturbo for state = 1:N
        C[state] = diag[state] * B[state]
    end
    for i = 1:L
        @tturbo for state = 1:N
            newstate = (state&(~(2^L))) ⊻ (1<<(i-1))
            c = newstate == 0
            newstate = !c*newstate + c*N # remove if statement
            C[newstate] -= B[state]
        end
    end
end
prepareHfunc!(L) = (C, B) -> Hfunc!(C, B, prepareDiag(L), L)

function f_LinearMap(L)
    H_LinearMap = LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
    d_LM, h_LM = partialschur(H_LinearMap, nev=1, which=SR())
    d_LM.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_exact (generic function with 1 method)

In [2]:
# https://juliaphysics.github.io/PhysicsTutorials.jl/tutorials/general/quantum_ising/quantum_ising.html

using LinearAlgebra
using SparseArrays
using ArnoldiMethod
⊗(x,y) = kron(x,y)

function TransverseFieldIsing_sparse(;N,h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    H = spzeros(Int, 2^N, 2^N) # note the spzeros instead of zeros here
    #for i in 1:N-1
    for i in 1:N # periodic boundary condition
        H -= foldl(⊗, first_term_ops)
        first_term_ops = circshift(first_term_ops,1)
    end
    
    for i in 1:N
        H -= h*foldl(⊗, second_term_ops)
        second_term_ops = circshift(second_term_ops,1)
    end
    H
end

function f_sparse(L)
    H = TransverseFieldIsing_sparse(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

f_sparse (generic function with 1 method)

In [3]:
f_exact(20)

-25.49098968636475

In [4]:
@time f_LinearMap(20)
@time f_LinearMap(20)

  8.097473 seconds (92.59 k allocations: 973.225 MiB, 2.82% gc time)
  8.033746 seconds (615 allocations: 968.031 MiB, 2.75% gc time)


-25.490989686364806 + 0.0im

In [5]:
@time f_sparse(20)
@time f_sparse(20)

 10.785176 seconds (4.72 k allocations: 7.038 GiB, 5.51% gc time)
 11.085410 seconds (4.72 k allocations: 7.038 GiB, 7.18% gc time)


-25.4909896863647 + 0.0im

In [6]:
@time H = TransverseFieldIsing_sparse(N=20, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.898631 seconds (4.66 k allocations: 6.718 GiB, 16.42% gc time)
  7.037959 seconds (243 allocations: 328.025 MiB, 0.34% gc time, 0.19% compilation time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.23608646317649218; -0.0648174167327405; … ; -0.0648174168866638; -0.23608646334949165;;], [-25.4909896863647;;], ComplexF64[-25.4909896863647 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [7]:
@time f_LinearMap(10)
@time f_LinearMap(10)

  0.004856 seconds (81 allocations: 665.359 KiB)
  0.003592 seconds (78 allocations: 664.953 KiB)


-12.784906442999326 + 0.0im

In [8]:
@time f_sparse(10)
@time f_sparse(10)

  0.005913 seconds (1.15 k allocations: 3.037 MiB)
  0.005827 seconds (1.15 k allocations: 3.037 MiB)


-12.784906442999327 + 0.0im

In [9]:
L = 10
@time H_LM = LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
@time d_LM, h_LM = partialschur(H_LM, nev=1, which=SR())
@time d_LM, h_LM = partialschur(H_LM, nev=1, which=SR())

  0.018664 seconds (25.96 k allocations: 1.478 MiB, 99.93% compilation time)
  0.009178 seconds (212 allocations: 672.875 KiB, 49.23% compilation time)
  0.003642 seconds (87 allocations: 666.062 KiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.11434387837217815; 0.11434387995844354; … ; 0.4105842730085417; 0.4105842766790208;;], [-12.78490644299935;;], ComplexF64[-12.78490644299935 + 0.0im]), Converged: 1 of 1 eigenvalues in 40 matrix-vector products)

In [10]:
@time H_sparse = TransverseFieldIsing_sparse(N=10, h=1)
@time d, h = partialschur(H_sparse; nev=1, which=SR())
@time d, h = partialschur(H_sparse; nev=1, which=SR())

  0.001126 seconds (1.11 k allocations: 2.705 MiB)
  0.004603 seconds (49 allocations: 341.438 KiB)
  0.002675 seconds (48 allocations: 341.141 KiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.41058427652986995; -0.11434388133853843; … ; -0.11434387984736308; -0.41058427284523213;;], [-12.784906442999334;;], ComplexF64[-12.784906442999334 + 0.0im]), Converged: 1 of 1 eigenvalues in 40 matrix-vector products)

In [16]:
using BenchmarkTools

L = 10
@show L
println("LinearMaps:")
H_LM = @btime LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
@btime d_LM, h_LM = partialschur($H_LM, nev=1, which=$(SR()))
println("SparseArrays:")
H_sparse = @btime TransverseFieldIsing_sparse(N=L, h=1)
@btime d, h = partialschur($H_sparse; nev=1, which=$(SR()));

L = 10
LinearMaps:
  223.396 ns (6 allocations: 128 bytes)
  3.401 ms (74 allocations: 664.31 KiB)
SparseArrays:
  697.200 μs (1113 allocations: 2.70 MiB)
  2.506 ms (36 allocations: 339.52 KiB)


In [17]:
using BenchmarkTools

L = 20
@show L
println("LinearMaps:")
H_LM = @btime LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
@btime d_LM, h_LM = partialschur($H_LM, nev=1, which=$(SR()))
println("SparseArrays:")
H_sparse = @btime TransverseFieldIsing_sparse(N=L, h=1)
@btime d, h = partialschur($H_sparse; nev=1, which=$(SR()));

L = 20
LinearMaps:
  216.667 ns (6 allocations: 128 bytes)
  7.770 s (621 allocations: 968.03 MiB)
SparseArrays:
  3.289 s (4664 allocations: 6.72 GiB)
  6.736 s (56 allocations: 328.01 MiB)
